In [9]:
import pandas as pd
# !pip install pandas_datareader
from pandas_datareader import data as web
from datetime import datetime
from datetime import date

In [10]:
#ler massa de dados
dreitr = pd.read_csv('DADOS/itr_cia_aberta_DRE_con_2011-2021')
dredfp = pd.read_csv('DADOS/dfp_cia_aberta_DRE_con_2011-2021')

# relação de empresas
empresas = dreitr[['DENOM_CIA','CD_CVM']].drop_duplicates().set_index('CD_CVM')
# Relação de contas
# contas = empresa[['CD_CONTA','DS_CONTA']].drop_duplicates().set_index('CD_CONTA')
# '3.99.01.01'


In [11]:
def CalculoLPA(codigo_cvm,conta_cvm='3.99.01.01'):

    #Selecao de empresas nos arquivos principais
    empresa = dreitr[dreitr['CD_CVM']==codigo_cvm]
    empresadfp = dredfp[dredfp['CD_CVM']==codigo_cvm]

    select_conta = conta_cvm
    #LPA do ITR
    lpa = empresa[empresa['CD_CONTA'] == select_conta]
    lpa = lpa[lpa['ORDEM_EXERC'] == 'ÚLTIMO']
    lpa['DT_INI_EXERC'] = pd.to_datetime(lpa['DT_INI_EXERC'])
    lpa['DT_FIM_EXERC'] = pd.to_datetime(lpa['DT_FIM_EXERC'])
    lpa['Timedelta'] = pd.to_timedelta(lpa['DT_FIM_EXERC']-lpa['DT_INI_EXERC'], unit='D', errors='raise') #calcular o delta dos registros para localizar os registros apenas de trimestres separados e não de acumulado do ano
    lpa = lpa[lpa['Timedelta'] < pd.Timedelta(100, unit='D')] #filtrar somente registros de trimestres em separado
    lpa = lpa[['DT_FIM_EXERC','VL_CONTA']].set_index('DT_FIM_EXERC')

    #LPA do DFP
    lpadfp = empresadfp[empresadfp['ORDEM_EXERC'] == 'ÚLTIMO']
    lpadfp = lpadfp[lpadfp['CD_CONTA'] == select_conta]
    lpadfp['DT_INI_EXERC'] = pd.to_datetime(lpadfp['DT_INI_EXERC'])
    lpadfp['DT_FIM_EXERC'] = pd.to_datetime(lpadfp['DT_FIM_EXERC'])
    lpadfp = lpadfp[['DT_FIM_EXERC','VL_CONTA']].set_index('DT_FIM_EXERC')
    lpa['ano'] = lpa.index.year #testar remover linha
    lpadfp['ano'] = lpadfp.index.year


    #Obter lpa somente do quarto tri
    lpaano = lpa.groupby('ano').sum()
    lpaano.rename(columns={'VL_CONTA':'VL_CONTA_AC_3Q'}, inplace=True)
    lpadfpano = lpadfp.groupby('ano').sum()
    lpadfpano.rename(columns={'VL_CONTA':'VL_CONTA_AC_4Q'}, inplace=True)

    lpa4q = lpaano.join(lpadfpano)
    lpa4q['VL_CONTA_SO_4Q'] = lpa4q['VL_CONTA_AC_4Q'] - lpa4q['VL_CONTA_AC_3Q']
    lpa4q.reset_index(inplace=True)
    lpa4q['date'] = lpa4q['ano'].astype(str)+'-12-31'
    lpa4q = lpa4q[['date','ano','VL_CONTA_SO_4Q']].set_index('date')
    lpa4q.rename(columns={'VL_CONTA_SO_4Q':'VL_CONTA'}, inplace=True)

    #Consolidar dataframe com LPAs por trimestre
    lpa_trimestral = pd.concat([lpa, lpa4q]).drop('ano',axis=1)
    lpa_trimestral.index = pd.to_datetime(lpa_trimestral.index)
    lpa_trimestral.sort_index(inplace=True)

    lpa_trimestral['ultimos 4q'] = lpa_trimestral['VL_CONTA'].rolling(4, min_periods=4).sum()
    lpa_trimestral['media 4q'] = lpa_trimestral['VL_CONTA'].rolling(4, min_periods=4).mean()


    # #Obter cotações
    # cotacoes = web.DataReader(f'ITSA4.SA', data_source='yahoo', start=lpa.index[0], end=lpa.index[-1])['Close']
    # cotacoes = pd.DataFrame(cotacoes)   

    # #Join de LPA x cotações e demais calculos
    # calculos = cotacoes.join(lpa_trimestral).ffill()

    # calculos['Close/Media'] = calculos['Close'] / calculos['media 4q']
    # calculos['Media/Close'] = calculos['media 4q'] / calculos['Close']


    # calculos['Close/ult4q'] = calculos['Close'] / calculos['ultimos 4q']
    # calculos['Close/ult4q'] = calculos['Close'] / calculos['ultimos 4q']
    # calculos['ult4q/Close'] = calculos['ultimos 4q'] / calculos['Close']


    return lpa_trimestral


In [12]:
# select_conta = '3.99.01.01'
# cod_cvm = 6505
# ticker = 'VIIA3.SA'


In [22]:
for empr in empresas.index:
    df = CalculoLPA(empr)
    df.to_excel('LPA/lpa_'+str(empr)+'.xlsx')




2437
14451
14540
14621
14796
15083
22594
15091
20990
15253
15369
15652
21733
15539
19887
16101
16330
16233
16225
16390
16292
19313
18708
16438
16497
18287
18678
16659
16624
19143
16632
16730
16993
16950
20800
17337
17361
18414
17450
17442
18660
17388
17329
17418
17930
17779
22551
17710
17639
17647
17655
17604
17671
17493
17558
17540
17922
17914
17892
17973
18112
18821
22454
18902
20575
19771
21431
20478
18716
18350
18376
19330
20010
19879
21415
19860
18635
19437
18651
18775
18724
20788
18953
19763
19089
19062
19100
18970
19453
18961
21237
21555
19305
20613
19402
21717
80004
80047
20354
22357
21300
19569
19917
19909
19925
20320
20338
19933
5991
20036
20087
20265
20966
20214
20982
20630
80020
20257
20419
20168
20435
20370
20400
20605
20770
20915
21490
4669
20540
21709
21636
21393
21539
21180
20850
20842
20907
21482
21008
21040
21016
21130
21903
80071
80098
21520
22438
80080
22560
21342
22110
22519
21660
21610
21792
21563
21512
21806
21652
22691
22144
22187
21776
22500
80110
22136
22756
2